In [3]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import operator
import math
import warnings
import openpyxl
import random
warnings.filterwarnings('ignore')
from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
import tensorflow as tf
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import pandas as pd

# 한글 글꼴체 변경
plt.rcParams['font.family'] ='Malgun Gothic'
# df.head() 이런거 했을 때, 컬럼이 생략되지 않고, 모든 컬럼 뜨게
pd.set_option('display.max_columns', None)
# 지수 표기법 대신에 소수점으로 표시하는코드
pd.options.display.float_format = '{:.2f}'.format
# 값 길이 제한 X
pd.set_option('display.max_colwidth', None) # 값 길이 제한 없음

### ✅ 신규 고객 구하는 코드
팀 회의 중, 첫번째 대시보드 화면에서 매 월 신규 고객을 파악했으면 좋겠다는 의견이 나와서 진행

In [4]:
df = pd.read_csv("data/full_data/TPS_cancel_data_Final.csv")

In [5]:
df

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn
0,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 12개월이상,Y,0.00,3개월내없음,N,10.00,HD,13,I/B,N,12개월~24개월미만,733,3.00,N,N,2,N
1,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,6.72,일주일내,N,0.00,HD,12,I/B,Y,24개월~ 36개월미만,764,3.00,N,Y,3,N
2,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,9.86,3주일전,N,10.00,HD,11,I/B,N,24개월~ 36개월미만,794,3.00,N,N,4,N
3,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 9~12개월,Y,5.95,4주일전,N,10.00,HD,10,I/B,N,24개월~ 36개월미만,825,3.00,N,N,5,N
4,0000113b86db7c509bbe74d609529031b03b7c033dbdfbd8b7fcecbf92bc8600,60대,약정만료전 6~9개월,Y,4.03,일주일내,N,0.00,HD,9,I/B,N,24개월~ 36개월미만,855,3.00,N,N,6,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21483374,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.03,일주일내,N,10.00,HD,-40,현장경로,N,36개월 이상,2338,1.00,N,N,8,N
21483375,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,2.91,일주일내,N,10.00,HD,-41,현장경로,N,36개월 이상,2368,1.00,N,N,9,N
21483376,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,1.97,2주일전,N,10.00,HD,-42,현장경로,N,36개월 이상,2399,1.00,N,Y,10,N
21483377,fffffa7eda8144ce27e65690933ae8994e6962fefd24c982990467add99d61a7,50대,약정만료후 12개월이상,N,0.00,3개월내없음,N,0.00,HD,-43,현장경로,N,36개월 이상,2429,1.00,N,N,11,N


각 sha2_hash별 p_mt의 최솟값이 '2'라면 신규 고객이 아니다.<br><Br>
p_mt의 최솟값이 '2'가 아닌 sha2_hash를 뽑아낸다.

In [6]:
# sha2_hash별 p_mt 최소값 계산
min_p_mt = df.groupby('sha2_hash')['p_mt'].min().reset_index()

# 최소값이 2가 아닌 sha2_hash 필터링 -> 2월달은 해당 데이터 중에서 신규 고객이 아님!
filtered_hashes = min_p_mt[min_p_mt['p_mt'] != 2]['sha2_hash']

In [7]:
display(filtered_hashes)

51         0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe
54         0001cc7febb2769a3f1992528464e8e5d7926571c4deee459048424385ca9091
82         00025086d582b6305b2e427822505d414a5e3920385be6638203ed14913c0a96
90         0002a102f968653d111bef88d347fa0e60397ffd47505adfaf1062bb0ea58718
122        0003b7afdc30ebda72b16f5e6917bf3bd10282f58470bb05055ae8ba9c0009ff
                                         ...                               
2075194    fffc5d8afdff0559a90f2011219c8cc7c453ae4f52c52a41bb0e5d0551e56d1c
2075206    fffcd8dd499cd07a4ffc74055f0ace40ed273eb26a477aef26a61a19948131cb
2075261    fffe8d7e5a58fdd2137b24156feccb91dff1904f877b617f0a01707b402d6a3d
2075262    fffea35fc5d88b89a8031befccb61a4f34b03b6d8e5e1f2e8f18747a73eab8db
2075292    ffff99d6fb91db75a5c5c18e4df7b6493049b4b84669bd944e0b1593930b1ed6
Name: sha2_hash, Length: 112875, dtype: object

In [9]:
# 해당 데이터는 5월에 신규 고객이라고 판단할 수 있다.
df[df['sha2_hash'] == '0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe'][['sha2_hash','p_mt']]

,sha2_hash,p_mt
561,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,5
562,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,6
563,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,7
564,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,8
565,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,9
566,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,10
567,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,11
568,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,12


p_mt의 최솟값이 '2'가 아닌 sha2_hash를 불러온다!

In [10]:
filtered_df = df[df['sha2_hash'].isin(filtered_hashes)]

In [11]:
filtered_df

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn
561,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,1.48,일주일내,N,0.00,HD,36,현장경로,Y,6개월미만,2672,3.00,N,Y,5,N
562,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,2.12,일주일내,N,10.00,HD,35,현장경로,N,6개월미만,2702,3.00,N,N,6,N
563,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,2.19,일주일내,N,10.00,HD,34,현장경로,N,6개월미만,2733,3.00,N,N,7,N
564,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,2.52,일주일내,N,10.00,HD,33,현장경로,N,6개월미만,2764,3.00,N,N,8,N
565,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,3.89,일주일내,N,20.00,HD,32,현장경로,N,6개월미만,2794,3.00,N,N,9,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21483209,ffff99d6fb91db75a5c5c18e4df7b6493049b4b84669bd944e0b1593930b1ed6,40대,약정만료전 12개월이상,N,12.54,일주일내,N,40.00,HD,34,현장경로,N,6개월미만,3415,2.00,N,N,8,N
21483210,ffff99d6fb91db75a5c5c18e4df7b6493049b4b84669bd944e0b1593930b1ed6,40대,약정만료전 12개월이상,N,9.48,일주일내,N,50.00,HD,33,현장경로,N,6개월미만,3445,2.00,N,N,9,N
21483211,ffff99d6fb91db75a5c5c18e4df7b6493049b4b84669bd944e0b1593930b1ed6,40대,약정만료전 12개월이상,N,7.99,일주일내,N,40.00,HD,32,현장경로,N,6개월미만,3476,2.00,N,N,10,N
21483212,ffff99d6fb91db75a5c5c18e4df7b6493049b4b84669bd944e0b1593930b1ed6,40대,약정만료전 12개월이상,N,4.47,일주일내,N,30.00,HD,31,현장경로,N,6개월미만,3506,2.00,N,N,11,N


이제, 각 sha2_hash별로 p_mt가 최솟값인 행을 구하면 해당 user가 유입된 월이 나오게 된다.

In [12]:
# 각 sha2_hash별로 p_mt가 최솟값인 행

min_p_mt_df = filtered_df.loc[filtered_df.groupby('sha2_hash')['p_mt'].idxmin()]

In [13]:
min_p_mt_df

,sha2_hash,AGE_GRP10,AGMT_END_SEG,BUNDLE_YN,CH_HH_AVG_MONTH1,CH_LAST_DAYS_BF_GRP,CONTENT_USE_YN,INHOME_RATE,MEDIA_NM_GRP,MONTHS_REMAINING,SCRB_PATH_NM_GRP,STB_RES_1M_YN,SVC_USE_DAYS_GRP,TOTAL_USED_DAYS,TV_I_CNT,VOC_STOP_CANCEL_MONTH1_YN,VOC_TOTAL_MONTH1_YN,p_mt,churn
561,0001ab8ff26ffa9d1904b8e1b02e1deee273b46aa5be8704f6123f6055aaa1fe,60대,약정만료전 12개월이상,Y,1.48,일주일내,N,0.00,HD,36,현장경로,Y,6개월미만,2672,3.00,N,Y,5,N
591,0001cc7febb2769a3f1992528464e8e5d7926571c4deee459048424385ca9091,60대,약정만료전 12개월이상,Y,0.07,일주일내,N,0.00,UHD,36,I/B,Y,6개월미만,6,4.00,N,Y,8,N
888,00025086d582b6305b2e427822505d414a5e3920385be6638203ed14913c0a96,20대,약정만료전 12개월이상,Y,0.00,3개월내없음,N,0.00,UHD,36,현장경로,Y,6개월미만,4,2.00,N,Y,10,N
968,0002a102f968653d111bef88d347fa0e60397ffd47505adfaf1062bb0ea58718,50대,약정만료전 12개월이상,N,5.55,일주일내,N,10.00,UHD,36,현장경로,Y,6개월미만,19,1.00,N,Y,5,N
1315,0003b7afdc30ebda72b16f5e6917bf3bd10282f58470bb05055ae8ba9c0009ff,50대,약정만료전 12개월이상,N,1.70,일주일내,Y,0.00,UHD,36,I/B,Y,6개월미만,1024,2.00,N,Y,6,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21482202,fffc5d8afdff0559a90f2011219c8cc7c453ae4f52c52a41bb0e5d0551e56d1c,40대,약정만료전 12개월이상,Y,2.39,일주일내,Y,0.00,UHD,36,현장경로,Y,6개월미만,1,2.00,N,Y,11,N
21482316,fffcd8dd499cd07a4ffc74055f0ace40ed273eb26a477aef26a61a19948131cb,50대,약정만료전 12개월이상,Y,5.16,일주일내,N,0.00,UHD,36,O/B,Y,6개월미만,5,3.00,N,Y,8,N
21482904,fffe8d7e5a58fdd2137b24156feccb91dff1904f877b617f0a01707b402d6a3d,60대,약정만료전 12개월이상,N,5.42,일주일내,N,10.00,UHD,36,현장경로,Y,36개월 이상,3576,2.00,Y,Y,11,N
21482906,fffea35fc5d88b89a8031befccb61a4f34b03b6d8e5e1f2e8f18747a73eab8db,50대,약정만료전 12개월이상,Y,3.49,일주일내,Y,0.00,UHD,36,현장경로,Y,6개월미만,11,2.00,N,Y,5,Y


In [14]:
min_p_mt_df['p_mt'].value_counts().sort_index()

p_mt
3     12167
4     10555
5     11490
6     12121
7     11109
8     11560
9     11640
10    11282
11    11266
12     9685
Name: count, dtype: int64

#### 신규 고객
3월 - 12,167명<br><br>
4월 - 10,555명<br><br>
5월 - 11,490명<br><br>
6월 - 12,121명<br><br>
7월 - 11,109명<br><br>
8월 - 11,560명<br><br>
9월 - 11,640명<br><br>
10월 - 11,282명<br><br>
11월 - 11,266명<br><br>
12월 - 9,685명<br><br>